# PHYS 126: Electrodynamics

## Preamble
By the end of this assignment, you will be able to simulate the motion of a small number of charged particles in electric and magnetic fields.

### Prerequisites: 
* Basic Python Introduction
* Kinematics
* Vectors
* Coulomb's law

### Computational topics:
* Plotting
* Functions
* Usage of Classes
* Iteration

### Specific tools used:
* min, max, abs
* numpy functions like sqrt, radians, sin, cos
* numpy.linspace
* matplotlib
  * pyplot, figure, savefig,
  * plot, title, x/ylabel, axis, legend
* python f-strings

In this assignment you will examine the motion of charged particles in electric and magnetic fields.  This will be done by plotting the coordinates and making small animations.  Specifically you will consider:
1) The motion of a charged particle in a static uniform external electric field.
1) The motion of a charged particle in a static non-uniform external electric field.
2) The motion resulting from two charged particles with no external electric field.
5) The motion of N particles with no external electric field.

## Setup
These import statements import modules that will be useful for this exercise.  It is good practice to put all import statements at the top of your program, to only import one thing per line, and to not rename the things you are importing.

In [ ]:
import os  # for manipulating files on the computer
import datetime  # to determine the date and time
import numpy as np  # to create and manipulate arrays of numbers
import matplotlib.pyplot as plt  # for plotting things
import random
#%matplotlib notebook

Now let's define some useful physical constants.

In [ ]:
c = 299792458  # m/s speed of light
mu0 = 1.25663706212e-6  # NA^2 magnetic constant
e0 = 1/(mu0*c*c)  # electric constant
k = 1/(4*np.pi*e0)  # Coulomb constant

Here we provide a _class_ to represent charged particles.  A class is a way of defining a new type of object.  For this assignment you don't need to understand how the class works, only how to use it.  The code is available for reading, but you only need to know how to create a new particle, ask for its the electric field, and move it according to a field.

In [ ]:
class Particle:
    """A Particle has a mass (in kg), charge (in C), a position vector (in m) and a velocity vector (in m/s).  
    
    The position and velocity can be 2D or 3D vectors, but they must be consistent, along with any
    external electric fields used in the move() method.
    
    Usage: my_particle = Particle(mass, charge, [x0, y0[, z0]], [vx0, vy0[, vz0]])
    """
    def __init__(self, mass, charge, position, velocity):
        """Create a new particle."""
        self.mass = float(mass)
        self.charge = float(charge)
        # The dtype = numpy.float64 options are to avoid accidentally creating
        # arrays of integers if you did e.g. position = [0, 0]
        self.position = np.array(position, dtype = np.float64)
        self.velocity = np.array(velocity, dtype = np.float64)

    def move(self, E_field, dt):
        """Given an external electric field, update the particle's motion after a time
        interval dt.
        """
        # Move the particle according to its existing velocity.
        self.position += self.velocity*dt
        
        # Calculate the electric force.
        E_force = self.charge*E_field
        acceleration = E_force/self.mass
        
        # Update the velocity based on the net acceleration.
        self.velocity += acceleration*dt

    def E_field(self, p):
        """Return the electric field produced by this particle at position p."""
        r = p - self.position  # Vector from particle to p
        r2 = np.dot(r, r)  # Get the squared magnitude of the vector r.
        rhat = r/np.sqrt(r2)  # Get a unit vector in the direction of r.
        return k*self.charge/r2 * rhat  # Coulomb's law

## Single particle in static uniform electric field
Start by creating a single particle with a given mass and charge, at rest at the origin.  We will only use 2D position and velocity vectors.

In [ ]:
mass = 1.0 # kg
charge = 1e-6 # 1 μC
particle = Particle(mass, charge, [0, 0], [0, 0])  # The [0, 0] are initial position and velocity.

Now we create a function that represents a uniform external electric field.  Electric fields are vectors so we need to return an array of two values since we are only dealing with 2D.

In [ ]:
def E_ext(p):
    """Return the value of this electric field (in N/C) at point p."""
    E0 = 1.0
    return np.array([E0, 0.0])

Now we are ready to simulate the motion of this particle.  We choose an iteration interval `dt` (in seconds) and a total simulation time.

In [ ]:
dt = 1  # Seconds
t0, t1 = 0, 1000  # Initial and final time of simulation
n_iters = int((t1-t0)/dt)  # Number of iterations
particle_coord = np.zeros( (n_iters, 2) )  # Create an n x 2 array of zeroes in which to put the coordinates.
times = np.zeros(n_iters)  # An array to store the individual time values.
t = 0  # Initialize time counter

for i in range(n_iters):
    pos = particle.position  # Get the position of the particle
    particle_coord[i] = pos  # Store the position in the coordinate array
    times[i] = t  # Store the time
    
    e_field = E_ext(pos)  # Calculate the e-field
    particle.move(e_field, dt)  # Update the position and velocity after time dt
    t += dt  # Update the time

What does this look like?  Let's plot it with matplotlib.  Note that `particle_coord` is an $n_{\text{iter}}\times 2$ array, but the `plot` command wants individual x- and y-value arrays, so we need to extract the values.

In [ ]:
x_values = particle_coord[:,0]
y_values = particle_coord[:,1]

fig1 = plt.figure()
plt.title("Motion of single particle in uniform e-field")

plt.plot(times, x_values, label="x")
plt.plot(times, y_values, label="y")
plt.xlabel("Time (s)")
plt.ylabel("Distance (m)")
plt.legend(loc="upper left")


Let's set up some infrastructure for saving plots.  It is good practice to put your plots or any other output files into a separate directory from your code.  Within python we can use the `os` module to create a subdirectory for our plots.  `exist_ok=True` tells it not to consider it an error if the directory already exists.  

In [ ]:
plotdir = "plots"
os.makedirs(plotdir, exist_ok=True)
fig1.savefig("plots/PHYS126_SingleParticle_Coordinates.png")

We can also plot y vs x to visualize the path instead of each coordinate as a function of time.  Since both axes will now represent metres, we should make a plot with matching range scaled.  That way a given visual distance on the x axis will mean the same thing as the same visual distance on the y axis.

In [ ]:
fig2 = plt.figure()
plt.title("Path of single particle in uniform e-field")
plt.axis("equal")  # This makes the axis scales match, so a size on screen is the same vertically or horizontally.
plt.plot(x_values, y_values, label="xy")
fig2.savefig("plots/PHYS126_SingleParticle_Path.png")

As expected, for a uniform electric field pointing in the x-direction, a positively charged particle starting at rest at the origin simply accelerates in the +x-direction while its y-coordinate stays at zero.

Task: Make a new figure showing the path of a particle in the same electric field, but starting with non-zero velocity $v_{\text{init}} = (-5.0, 5.0)\times 10^{-4} \text{m/s}$.  You will need to fill out the parts of the code where we left `...`.

In [ ]:
############
# SOLUTION #
############
v_init = [-5e-4, 5e-4]
particle = Particle(mass, charge, [0, 0], v_init)  # Add a reasonable non-zero initial velocity
particle_coord = np.zeros( (n_iters, 2) )  # Create an n x 2 array of zeroes in which to put the coordinates.
times = np.zeros(n_iters)  # An array to store the individual time values.
t = 0  # Initialize time counter

for i in range(n_iters):
    pos = particle.position  # Get the position
    particle_coord[i] = pos  # Store the position in the coordinate array before updating.
    times[i] = t  # Store the time

    e_field = E_ext(pos)  # Calculate the e-field
    particle.move(e_field, dt)  # Update the position and velocity after time dt
    t += dt
    
x_values = particle_coord[:,0]
y_values = particle_coord[:,1]

fig3 = plt.figure()
plt.title(f"Path of particle in uniform e-field v_init={v_init} m/s")
plt.axis("equal")  # This makes the axis scales match, so a size on screen is the same vertically or horizontally.
plt.plot(x_values, y_values, label="xy")
fig3.savefig("plots/PHYS126_SingleParticle_VInit.png")

In [ ]:
v_init = ...  # Write the initial velocity as a 2-element sequence.
particle = ...  # Create a new particle
particle_coord = ...  # Create an n x 2 array of zeroes in which to put the coordinates.
times = ...  # An array to store the individual time values.
t = 0  # Initialize time counter

for i in range(n_iters):
    ...  # Get the position
    ...  # Store the position in the coordinate array
    ...  # Store the time

    ...  # Calculate the e-field
    ...  # Update the position and velocity after time dt
    t += dt
    
x_values = ...  # Extract the x-values from the coordinate array
y_values = ...  # Extract the y-values from the coordinate array

fig3 = ...  # Make a figure object
...  # Give the figure a title
...  # Make the axis scales match, so a size on screen is the same vertically or horizontally.
...  # Plot the data
...  # Save the figure as PHYS126_SingleParticle_VInit.png in your plots directory

### Non-uniform electric field

Here we now provide a more complicated function `E_ext(p)` that returns a position-dependent electric field.  The new field behaves as $\vec{E}(\vec{p}) = E_0 \left(\sin(k_x x+\Delta_x)\hat{x} - \log\left|k_y y+\Delta_y\right|\hat{y}\right)$ for $E_0 = 100 \text{N/C}$, $k_x = 10 \text{m}$, $k_y = 1.0 \text{m}$, and $\Delta_x = \Delta_y = 0.01 \text{m}$.

In [ ]:
def E_ext(p):
    """Return the value of this electric field (in N/C) at point p."""
    E0 = 100.0
    kx = 10
    ky = 1.0
    dx = dy = 0.01
    x_comp = E0*np.sin(kx*p[0]+dx)
    y_comp = -E0*np.log(abs(ky*p[1]+dy))
    return np.array([x_comp, y_comp])

Task: Based on the code for the example with the uniform electric field, make a plot of the path (y vs. x) of a charged particle in the new non-uniform field, with initial velocity $v_{\text{init}} = (1.0, 1.0)\times 10^{-4} \text{m/s}$.  Save the plot to a file called `PHYS126_NonUniformField.png`.

In [ ]:
############
# SOLUTION #
############
# Re-use mass, charge, and time variables (t0, t1, dt, n_iters) from example 1.

particle = Particle(mass, charge, [0, 0], [1e-4, 1e-4])

# Make new arrays
particle_coord = np.zeros( (n_iters, 2) )  # Create an n x 2 array of zeroes in which to put the coordinates.
times = np.zeros(n_iters)  # An array to store the individual time values.
t = 0  # Initialize time counter

for i in range(n_iters):
    pos = particle.position  # Get the position
    particle_coord[i] = pos  # Store the position in the coordinate array before updating.
    times[i] = t  # Store the time

    e_field = E_ext(pos)  # Calculate the e-field and zero b-field
    particle.move(e_field, dt)  # Update the position and velocity after time dt
    t += dt

x_values = particle_coord[:,0]
y_values = particle_coord[:,1]

fig4 = plt.figure()
plt.title("Path of single particle in non-uniform e-field")
plt.axis("equal")  # This makes the axis scales match, so a size on screen is the same vertically or horizontally.
plt.plot(x_values, y_values, label="xy")
fig4.savefig("plots/PHYS126_NonUniformField.png")

In [ ]:
...  # Write the initial velocity as a 2-element sequence.
...  # Create a new particle
...  # Create an n x 2 array of zeroes in which to put the coordinates.
...  # An array to store the individual time values.
t = 0  # Initialize time counter

for i in range(n_iters):
    ...  # Get the position
    ...  # Store the position in the coordinate array
    ...  # Store the time

    ...  # Calculate the e-field
    ...  # Update the position and velocity after time dt
    t += dt
    
x_values = ...  # Extract the x-values from the coordinate array
y_values = ...  # Extract the y-values from the coordinate array

fig4 = ...  # Make a figure object
...  # Give the figure a title
...  # Make the axis scales match, so a size on screen is the same vertically or horizontally.
...  # Plot the data
...  # Save the figure as PHYS126_NonUniformField.png in your plots directory

## Two particles
Now we will simulate the effect of two particles interacting, with zero external field.  To observe non-trivial behaviour, one of the particles will start with non-zero initial velocity.

In [ ]:
charge = 1.0e-6  # A more realistic charge for two particles interacting metres apart.

# Set up some initial positions and velocities.
p1_start = [1.0, 0.0]
p1_v0 = [0.0, 0.1]
p2_start = [-1.0, 0.0]
p2_v0 = [0.0, -0.0]
p1 = Particle(mass, charge, p1_start, p1_v0)
p2 = Particle(mass, -charge, p2_start, p2_v0)  # One particle has negative charge so they are attracting

dt = 1  # Seconds
t0, t1 = 0, 1000  # Initial and final time of simulation
n_iters = int((t1-t0)/dt)  # Number of iterations

# Arrays to store the simulation values, this time we have two particle coordinate arrays.
p1_coord = np.zeros((n_iters, 2))
p2_coord = np.zeros((n_iters, 2))
times = np.zeros(n_iters) 
t = 0  # Initialize time counter

for i in range(n_iters):
    pos1 = p1.position  # Get the positions
    pos2 = p2.position

    # Store the positions in the coordinate arrays before updating.
    p1_coord[i] = pos1
    p2_coord[i] = pos2
    times[i] = t  # Store the time

    # Calculate the fields at each of the particles' positions
    E12 = p1.E_field(pos2)  # Field at p2 from p1's charge
    E21 = p2.E_field(pos1)  # Field at p1 from p2's charge
        
    # Update the position and velocity after time dt
    # Note p1 is affected by field E21 (field at p1 from p2's charge) and vice-versa.
    p1.move(E21, dt)
    p2.move(E12, dt)
    t += dt

Plotting is similar to what we did with one particle.  Here we only plot the paths (y vs. x), skipping the single-coordinate plots.

In [ ]:
p1_x = p1_coord[:,0]
p1_y = p1_coord[:,1]

p2_x = p2_coord[:,0]
p2_y = p2_coord[:,1]

fig5 = plt.figure()
plt.title("Two Particles Interacting")
plt.axis("equal")  # This makes the axis scales match, so a size on screen is the same vertically or horizontally.
plt.plot(p1_x, p1_y, label="p1")
plt.plot(p2_x, p2_y, label="p2")
plt.legend()
fig5.savefig("plots/PHYS126_TwoParticles.png")

Task: Recreate the two-particle simulation with initial values so that you get a reasonably stable "orbit".  Hint: remember Kepler's laws of planetary motion.

## Discussion of Results
Write your responses to the following questions with a `>` at the start of each line to create a Markdown "blockquote" like this:

> This is my response.

1. How do we know we made a good choice for the time interval `dt` and `t1`?  What happens if you try to set `dt` to be much smaller for a more accurate simulation?
2. What happens if you change the initial velocities for figures 3 and 4?
3. Describe conceptually how you could extend this simple simulation to include an arbitrary number of particles.
4. Describe conceptually how you could extend this simple simulation to include external magnetic fields.
3. What were the easiest and most difficult parts of this assignment?  
4. Name another physical system that you feel you could simulate, having completed this assignment.

## Submission and Grading
Submit a single-file Jupyter notebook named PHYS126_Electrodynamics_SFUID.ipynb  This file will be run by the grading TA and it is expected to produce the five figures.  We will look for:
1. The code runs without raising exceptions or crashing.
2. Your code is readable and has useful comments throughout, without unnecessary comments for trivial operations.
4. All plots have proper axis labels, with units.
5. The plots generally contain the expected data.
6. You have written responses to the Discussion questions at the bottom of your code, in plain english using complete sentences.